In [1]:
from qm.qua import *
from qm import QuantumMachinesManager
from qm import SimulationConfig, LoopbackInterface
from qualang_tools.loops import from_array
from qualang_tools.results import fetching_tool, progress_counter
from qualang_tools.plot import plot_demodulated_data_2d, interrupt_on_close,plot_demodulated_data_1d
from qualang_tools.units import unit
from configuration import *
import matplotlib.pyplot as plt
from datetime import date,datetime
from scipy import signal
from scipy.signal import find_peaks
import os
import pickle as pk
import sys
import plotly.io as pio
pio.renderers.default='browser'

qmm = QuantumMachinesManager(host='172.16.33.107')

2025-01-10 11:55:20,401 - qm - INFO     - Starting session: 71573114-0a2f-4002-b6b1-f288416bb19f
2025-01-10 11:55:24,982 - qm - INFO     - Performing health check
2025-01-10 11:55:25,244 - qm - INFO     - Cluster healthcheck completed successfully.


In [ ]:
sys.path.append(r'C:\Users\Marco\Documents\control-core\control_core\util\external')
import pilpxi_wrapper
import ctypes as ct
# SET ATT
bus = 36
slot = 15
ATT = pilpxi_wrapper.pilpxi_card(bus, slot)


# Flux map

We perform spectroscopy measurements of the our device. The device has two loops so two fluxes should be provided. The first ( we call it fast flux or phi+) is generated the "fast_flux_line" element. The second flux is a combination of the signals generated by "dc_flux_line" and "dc_flux_line2". The weight of the linear combination is adjusted by the alfa parameter

In [ ]:
#ATT.SetAttenuation(6, ct.c_float(3))
from configuration import *
# timings
readout_delay = 16 * u.ns
cooldown_time = 100 * u.ns

simulation = False

###################
# The QUA program #  ! time unit is 4ns !
###################

# Scan parameters
n_avg = 10
f_min = 62.5e6
f_max = 62.5e6
df = 1e6
if_freq = 35e6# np.array([20e6])#np.arange(f_min, f_max, df)

flux_min = -0.3
flux_max = 0.3
dflux = 0.0025
dc_flux = np.arange(flux_min, flux_max, dflux)

flux_min = -0.3
flux_max = 0.3
dflux = 0.005
fast_flux = np.arange(flux_min, flux_max, dflux)
alfa=1.0

with program() as hello_qua:
    # QUA variables declaration
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()
    f = declare(int)
    qua_fast_flux = declare(fixed)
    qua_dc_flux = declare(fixed)
    n = declare(int)
    n_st = declare_stream()

    # Scan a parameter
    with for_(n, 0, n < n_avg, n+1):
        update_frequency("resonator", if_freq) #takes 62 clock cycles
        with for_(*from_array(qua_fast_flux, fast_flux)):
            set_dc_offset("fast_flux_line", "single", qua_fast_flux)
            with for_(*from_array(qua_dc_flux, dc_flux)):
                set_dc_offset("dc_flux_line", "single", qua_dc_flux )
                set_dc_offset("dc_flux_line2", "single", qua_dc_flux * alfa )

                reset_global_phase() #takes 11 cycles
                reset_frame("resonator") #takes 4 cycles

                align()  # Make sure all pulses have been played before measuring
                wait(readout_delay * u.ns, "resonator")  # Wait some time after setting the dc flux

                # Measure using dual demodulation
                measure(
                    "readout",
                    "resonator",
                    None,
                    demod.full("cos", I, "out1"),
                    demod.full("sin", Q, "out1"),
                )
                # End of the sequence
                wait(cooldown_time * u.ns)
                # Save the data
                save(I, I_st)
                save(Q, Q_st)
        save(n, n_st)

    # Transfer the data from the FPGA to the CPU and perform some operations on the way (buffering, averaging, FFT...)
    with stream_processing():
        I_st.buffer(len(dc_flux)).buffer(len(fast_flux)).average().save("I")
        Q_st.buffer(len(dc_flux)).buffer(len(fast_flux)).average().save("Q")
        n_st.save("iteration")

if simulation:
    simulation_config = SimulationConfig(duration=12000//4)
    job = qmm.simulate(config, hello_qua, simulation_config)
    plt.figure()
    samples = job.get_simulated_samples()
    samples.con1.plot()
    # plt.legend(("stimulus_I", "stimulus_Q", "readout_I", "readout_Q", "fast_flux", "dc_flux"))
    plt.show()
    waveform_report = job.get_simulated_waveform_report()
    waveform_report.create_plot(samples, plot=True)

else:
    qm = qmm.open_qm(config)
    job = qm.execute(hello_qua)
    #qm.close()
    # Get results from QUA program
    results = fetching_tool(job, data_list=["I", "Q", "iteration"], mode="live")
    # Live plotting
    #interrupt_on_close(fig, job)
    while results.is_processing():
        # Fetch results
        I, Q, iteration = results.fetch_all()
        # Progress bar
        progress_counter(iteration, n_avg, start_time=results.start_time)
        # Live plot
        #Saving data
        data = {}
        data['config'] = config
        data['Q'] = Q
        data['I'] = I
        data['amp'] = np.sqrt(I**2 + Q**2)
        data['phase'] = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
        data['freq'] =  (if_freq+resonator_LO)*1e-9
        data['vpp'] = fast_flux
        data['vpms'] = dc_flux
        data['n_avg'] = n_avg
        data['readout_delay'] = readout_delay
        data['cooldown'] = cooldown_time

In [ ]:
z_val = data['I']
plt.figure()
plt.pcolormesh(data['vpms'],data['vpp'],z_val)
plt.colorbar()
plt.xlabel('Vpm')
plt.ylabel('Vpp')
plt.axvline(x = -0.023,color = 'k')
plt.axhline(y = 0.072,color = 'r')
plt.axhline(y = -0.082,color = 'r')

# Double well experiment

In [ ]:
%matplotlib qt

data = job.get_simulated_samples().con1.analog
plt.figure()
plt.plot(data['1-1'],label = 'in')
plt.plot(data['3'],label = 'RO')
plt.plot(data['5'],label = 'FFl')
plt.plot(data['6'],label = 'dc1')
plt.axvline(x=256,color='k')
plt.legend()

This is the double Well experiment necessary to setup the JDPD before phase detection. More details are reported in https://ieeexplore.ieee.org/abstract/document/10402570

In [ ]:
from configuration import *

vpp_0 = -0.082
vpp_pi = 0.072
flux_off =-0.06
flux_min = -0.05
flux_max  = 0.05
vpm_points = 101
alfa = 1.02

signs= [-1,1]

start_cooldown_time = 100 #in nanoseconds
input_duration = 200 #in nanosecs
tilt_delay = 100
flip_delay = 4
reset_time = 100

simulation = False

for sgn in signs:

    #PREPARING ARRAY OF SWEEP
    if sgn==1:
        flux = np.linspace(flux_min, flux_max, vpm_points)  + flux_off
        print(sgn,flux)
    else:
        flux = np.linspace(flux_max, flux_min, vpm_points) +  flux_off
        print(sgn,flux)
    flux = list(flux)
    flux_start = flux[0]

    with program() as hello_qua:
        # QUA variables declaration
        I = declare(fixed)
        Q = declare(fixed)
        I_st = declare_stream()
        Q_st = declare_stream()
        vpm = declare(fixed)
        with for_(*from_array(vpm, flux)):
            reset_global_phase()
            align()
            wait(reset_time * u.ns)
            wait(start_cooldown_time * u.ns, "stimulus")
        
            play("const"*amp(vpp_0 * 4), "fast_flux_line")
            play("const"*amp(flux_start * 4), "dc_flux_line")
            play("const"*amp(flux_start * 4*alfa), "dc_flux_line2")
            play("cw"*amp(0), "stimulus")
        
            #after play it means delay keeping the same value, than it plays for 16 ( the dur set in the config)
            wait((start_cooldown_time + input_duration -16 - flip_delay) * u.ns, "fast_flux_line")
            wait((start_cooldown_time + input_duration + tilt_delay -16) * u.ns, "dc_flux_line")
            wait((start_cooldown_time + input_duration + tilt_delay - 16) * u.ns, "dc_flux_line2")      
        
            play("const"*amp((vpp_pi - vpp_0)  * 4), "fast_flux_line")
            play("const"*amp((vpm - flux_start) * 4), "dc_flux_line")
            play("const"*amp((vpm - flux_start) * 4*alfa), "dc_flux_line2")
        
            wait((100*readout_len + flip_delay + 50 - 16) * u.ns, "fast_flux_line")
            wait((100*readout_len - tilt_delay + 50 - 16) * u.ns, "dc_flux_line")
            wait((100*readout_len - tilt_delay + 50 - 16) * u.ns, "dc_flux_line2")
        
            # Measurement
            wait((start_cooldown_time + input_duration + tilt_delay+16) * u.ns, "resonator")  # Wait some time after setting the dc flux

            # Measure using dual demodulation
            measure(
                "readout",
                "resonator",
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            align("fast_flux_line","dc_flux_line","dc_flux_line2","resonator")
            # End of the sequence
            ramp_to_zero("fast_flux_line")
            ramp_to_zero("dc_flux_line")
            ramp_to_zero("dc_flux_line2")
            # Save the data
            save(I, I_st)
            save(Q, Q_st)
            
        # Transfer the data from the FPGA to the CPU and perform some operations on the way (buffering, averaging, FFT...)
        with stream_processing():
             I_st.buffer(len(flux)).save("I")
             Q_st.buffer(len(flux)).save("Q")
    
    if simulation:
        simulation_config = SimulationConfig(duration=4000//4)
        job = qmm.simulate(config, hello_qua, simulation_config)
        plt.figure()
        job.get_simulated_samples().con1.plot()
        # plt.legend(("stimulus_I", "stimulus_Q", "readout_I", "readout_Q", "fast_flux", "dc_flux"))
        
    else:
        qm = qmm.open_qm(config,close_other_machines=False)
        job = qm.execute(hello_qua)
        results = fetching_tool(job, data_list=["I", "Q"], mode="wait_for_all")
        res = results.fetch_all()
            
        I,Q= np.array(res)
        amp_res = np.sqrt(I**2 + Q**2)
        plt.figure(1)
        plt.title('amps')
        plt.plot(flux,amp_res,'o-',label = 'dir is %f'%(sgn))
        plt.legend()

        phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
        plt.figure(2)
        plt.title('phase')
        plt.plot(flux,phase,'o-',label = 'dir is %f'%(sgn))
        plt.legend()

        plt.figure(3)
        plt.title('I')
        plt.plot(flux,I,'o-',label = 'dir is %f'%(sgn))
        plt.legend()

        plt.figure(4)
        plt.title('Q')
        plt.plot(flux,Q,'o-',label = 'dir is %f'%(sgn))

        plt.legend()
        qm.close()

In [ ]:
qm.close()

# Zero point

This is the zero point preparation necessary to find the JDPD working point before phase detection. More details are reported in https://ieeexplore.ieee.org/abstract/document/10402570

In [ ]:
from configuration import *

start_cooldown_time = 100 #in nanoseconds
input_duration = 100 #in nanosecs
tilt_delay = 24
readout_duration = 200
flip_delay = 4
reset_time = 16
vpp_0 = -0.082
vpp_pi = 0.072
n_avg = 1
alfa = 1.02

flux_min= -0.08
flux_max  = -0.05
vpm_points = 31

meas_point = -0.05
n_avg = 101
flux = np.linspace(flux_min, flux_max, vpm_points)

simulation = False

with program() as hello_qua:
    # QUA variables declaration
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()
    vpm = declare(fixed)
    n = declare(int)
    with for_(*from_array(vpm, flux)):
        with for_(n, 0, n < n_avg, n+1):
            reset_global_phase()
            align()
    
            wait(reset_time * u.ns)
    
            wait(start_cooldown_time * u.ns, "stimulus")
        
            play("const"*amp(vpp_0 * 4), "fast_flux_line")
            play("const"*amp(vpm* 4), "dc_flux_line")
            play("const"*amp(vpm * 4*alfa), "dc_flux_line2")
            play("cw"*amp(0), "stimulus")
        
            #after play it means delay keeping the same value, than it plays for 16 ( the dur set in the config)
            wait((start_cooldown_time + input_duration -16 - flip_delay) * u.ns, "fast_flux_line")
            wait((start_cooldown_time + input_duration + tilt_delay -16) * u.ns, "dc_flux_line")
            wait((start_cooldown_time + input_duration + tilt_delay - 16) * u.ns, "dc_flux_line2")     
        
            play("const"*amp((vpp_pi - vpp_0) * 4), "fast_flux_line")
            play("const"*amp((meas_point - vpm) * 4), "dc_flux_line")
            play("const"*amp((meas_point - vpm) * 4*alfa), "dc_flux_line2")
        
            wait((readout_duration + flip_delay+ 50 - 16) * u.ns, "fast_flux_line")
            wait((readout_duration -tilt_delay + 50 - 16) * u.ns, "dc_flux_line")
            wait((readout_duration -tilt_delay+ 50 - 16) * u.ns, "dc_flux_line2")
        
            # Measurement
            wait((start_cooldown_time + input_duration + tilt_delay+16) * u.ns, "resonator")  # Wait some time after setting the dc flux
            # Measure using dual demodulation
            measure(
                "readout",
                "resonator",
                None,
                dual_demod.full("cos", "out1", "sin", "out2", I),
                dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            )
            align("fast_flux_line","dc_flux_line","dc_flux_line2","resonator")
            # End of the sequence
            ramp_to_zero("fast_flux_line")
            ramp_to_zero("dc_flux_line")
            ramp_to_zero("dc_flux_line2")
            # Save the data
            save(I, I_st)
            save(Q, Q_st)
            # Transfer the data from the FPGA to the CPU and perform some operations on the way (buffering, averaging, FFT...)
    with stream_processing():
            
         I_st.buffer(n_avg).buffer(len(flux)).save("I")
         Q_st.buffer(n_avg).buffer(len(flux)).save("Q")

if simulation:
    simulation_config = SimulationConfig(duration=4000//4)
    job = qmm.simulate(config, hello_qua, simulation_config)
    %matplotlib qt5
    plt.close('all')
    data = job.get_simulated_samples().con1.analog
    plt.figure()
    plt.plot(data['1-1'],label = 'in')
    plt.plot(data['3'],label = 'RO')
    plt.plot(data['5'],label = 'FFl')
    plt.plot(data['6'],label = 'dc1')
    plt.axvline(x=256,color='k')
    plt.legend()

else:
    %matplotlib inline
    qm1 = qmm.open_qm(config, close_other_machines= False)
    job = qm1.execute(hello_qua)
    results = fetching_tool(job, data_list=["I", "Q"], mode="wait_for_all")
    res = results.fetch_all()
    qm1.close()


In [ ]:
res2 = np.array(res)
I_tot,Q_tot= np.array(res2)
#I_tot = np.transpose(I_tot)
#Q_tot = np.transpose(Q_tot)72
thrs_I_tot= (np.max(I_tot) + np.min(I_tot))/2
thrs_Q_tot = (np.max(Q_tot) + np.min(Q_tot))/2
plt.figure(1)
prob = []
for i,start_point in enumerate(flux):
    x = np.ones(n_avg) * start_point

    I = I_tot[i]
    Q = Q_tot[i]
    plt.figure(1)
    plt.title('Q')
    plt.plot(x,Q,'o',color = 'k')
    #prob.append(len(np.where(Q>thrs_Q_tot)[0])/len(Q))

    
    plt.figure(2)
    plt.plot(x,I,'o',color = 'k')
    prob.append(len(np.where(I>thrs_I_tot)[0])/len(Q))
    
    phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
    plt.figure(3)
    plt.plot(x,phase,'o',color = 'k')
    plt.legend()

plt.figure(5)
plt.plot(flux, prob, 'o-')
plt.axvline(x = -0.067,color = 'r')
plt.axhline(y = 0.5,color = 'r')

In [ ]:
data  = {}
data['results'] = prob_max 
data['phim_flux '] = flux 
data['reps'] = n_avg
data['attenuation'] = attenuations
data['LO_freq'] = 500e6
data['LO_pwr'] = -10

import pickle 
folder = r'C:\Users\Marco\Box\Seeqc - Readout\SFQ_readout_project\Measurements\R201\OPX'
os.chdir(folder)
with open('R201_15,11_JDPD1_700pH_zero_point_vs_LO-noise.pickle', 'wb') as file:
    # Use pickle.dump() to write the object to the file
    pickle.dump(data, file)

# Phase detect

In [ ]:
ATT.SetAttenuation(2, ct.c_float(12))
ATT.SetAttenuation(6, ct.c_float(9))

Phase detection experiment. It's almost the same experiment of the zero point but we include the input microwave tone of which we would like to measure the phase. In this particular experiment, we generate I,Q signals for mixing with an external LO. With the OPX 1000, I,Q signals should be replaced by a unique microwave tone. We should sweep over the phase of this tone.

In [52]:
from configuration import *

# ATT.SetAttenuation(2, ct.c_float(18))

vpp_0 = -0.04
vpp_pi = 0.072
alfa = 1.02

phase_start= 0
phase_stop  = 0.5
phase_points = 2
input_amp= 0.2

LO_input_freq = 4e9

if_freq = 100e6
start_cooldown_time = 100 #in nanoseconds
input_duration = 200 #in nanosecs
tilt_delay = 100
readout_duration = 2000
flip_delay = 40
reset_time = 1000 
sequence_duration = 1.6e4 - 128
sequence_duration_2 = 1.6e4 - 36

zero_point = -0.0635
meas_point = -0.05
n_avg = 501
input_phase = np.linspace(phase_start, phase_stop, phase_points)
# input_phase = np.array([0.5,0.5])
simulation = False

with program() as hello_qua:
    # QUA variables declaration
    I = declare(fixed)
    Q = declare(fixed)
    I_st = declare_stream()
    Q_st = declare_stream()
    phase = declare(fixed)
    n = declare(int)

    update_frequency("stimulus", if_freq)
    #with for_(*from_array(phase, input_phase)):   
        #wait(sequence_duration_2 * u.ns)
    with infinite_loop_():
        play("const"*amp(vpp_0 * 4), "fast_flux_line", duration=16)
        play("const"*amp(zero_point* 4), "dc_flux_line", duration=16)
        play("const"*amp(zero_point * 4*alfa), "dc_flux_line2", duration=16)
        reset_global_phase()
        with for_(*from_array(phase, input_phase)):  
        # with for_(n, 0, n < n_avg, n+1):
                
#            wait(reset_time * u.ns)
            
            # wait(sequence_duration * u.ns)
            

            # reset_frame("stimulus")
            # frame_rotation_2pi(phase, 'stimulus')
            fast_frame_rotation([[phase,0],[0,phase]],'stimulus')
            # align("stimulus","fast_flux_line","dc_flux_line","dc_flux_line2","resonator")

    
            # wait(start_cooldown_time * u.ns, "stimulus")
        
            # play("const"*amp(vpp_0 * 4), "fast_flux_line", duration=16)
            # play("const"*amp(zero_point* 4), "dc_flux_line", duration=16)
            # play("const"*amp(zero_point * 4*alfa), "dc_flux_line2", duration=16)
            play("cw"*amp(input_amp*4), "stimulus", duration=16*u.ns)
                    
            #after play it means delay keeping the same value, than it plays for 16 ( the dur set in the config)
            # wait((start_cooldown_time -16) * u.ns, "stimulus")
            # wait((start_cooldown_time + input_duration -16 - flip_delay) * u.ns, "fast_flux_line")
            # wait((start_cooldown_time + input_duration + tilt_delay -16) * u.ns, "dc_flux_line")
            # wait((start_cooldown_time + input_duration + tilt_delay - 16) * u.ns, "dc_flux_line2")
                
            # play("cw"*amp(input_amp), "stimulus")
            # play("const"*amp((vpp_pi - vpp_0) * 4), "fast_flux_line", duration=16)
            # play("const"*amp((meas_point - zero_point) * 4), "dc_flux_line", duration=16)
            # play("const"*amp((meas_point - zero_point) * 4*alfa), "dc_flux_line2", duration=16)
    
            #wait((readout_duration + 50 - 16) * u.ns, "stimulus")
            # wait((100*readout_duration + flip_delay+ 50 - 16) * u.ns, "fast_flux_line")
            # wait((100*readout_duration -tilt_delay + 50 - 16) * u.ns, "dc_flux_line")
            # wait((100*readout_duration -tilt_delay+ 50 - 16) * u.ns, "dc_flux_line2")
        
            # Measurement
            # wait((start_cooldown_time + input_duration + tilt_delay+16) * u.ns, "resonator")  # Wait some time after setting the dc flux
            # Measure using dual demodulation
            # measure(
            #     "readout",
            #     "resonator",
            #     None,
            #     dual_demod.full("cos", "out1", "sin", "out2", I),
            #     dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
            # )
            # align("fast_flux_line","dc_flux_line","dc_flux_line2","resonator")
            # End of the sequence
        wait(100)
        ramp_to_zero("fast_flux_line")
        ramp_to_zero("dc_flux_line")
        ramp_to_zero("dc_flux_line2")
            # Save the data
    #         save(I, I_st)
    #         save(Q, Q_st)
    #         # Transfer the data from the FPGA to the CPU and perform some operations on the way (buffering, averaging, FFT...)
    # with stream_processing():
        
    #     I_st.buffer(n_avg).buffer(len(input_phase)).save("I")
    #     Q_st.buffer(n_avg).buffer(len(input_phase)).save("Q")

if simulation:
    simulation_config = SimulationConfig(duration=100000//4)
    job = qmm.simulate(config, hello_qua, simulation_config)
    %matplotlib qt
    plt.close('all')
    data = job.get_simulated_samples().con1.analog
    job.get_simulated_samples().con1.plot()
    # plt.figure()
    # plt.plot(data['1-1'],label = 'in')
    # plt.plot(data['2'],label = 'in_Q')
    # plt.plot(data['3'],label = 'RO_1')
    # plt.plot(data['4'],label = 'RO_Q')
    # plt.plot(data['5'],label = 'FFl')
    # plt.plot(data['6'],label = 'dc1')
    # #plt.axvline(x=256,color='k')
    # plt.legend()
    plt.show()
    
else:
    %matplotlib inline
    qm1 = qmm.open_qm(config, close_other_machines= False)
    job = qm1.execute(hello_qua)
    # results = fetching_tool(job, data_list=["I", "Q"], mode="wait_for_all")
    # res = results.fetch_all()
    # qm1.close()

QmQuaException: Can't handle [[<qm.qua._expressions.QuaVariable object at 0x142ace010>, 0], [0, <qm.qua._expressions.QuaVariable object at 0x142ace010>]]

In [51]:
qmm.close_all_qms()

In [ ]:
res2 = np.array(res)
I_tot,Q_tot= np.array(res2)
#I_tot = np.transpose(I_tot)
#Q_tot = np.transpose(Q_tot)72
thrs_I_tot= (np.max(I_tot) + np.min(I_tot))/2
thrs_Q_tot = (np.max(Q_tot) + np.min(Q_tot))/2
plt.figure(1)
prob = []
for i,start_point in enumerate(input_phase):
    x = np.ones(n_avg) * start_point

    I = I_tot[i]
    Q = Q_tot[i]
    plt.figure(1)
    plt.plot(x,Q,'o',color = 'k')
    prob.append(len(np.where(I>thrs_I_tot)[0])/len(Q))

    
    plt.figure(2)
    plt.plot(x,I,'o',color = 'k')
    
    phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
    #plt.figure(3)
    #plt.plot(x,phase,'o',color = 'k')
    #plt.legend()

plt.figure(5)
plt.plot(input_phase, prob, 'o-')
#plt.axvline(x = 0.0213,color = 'r')


# Qubit RO with JDPD

In [ ]:
from configuration import *

vpp_0 = -0.11
vpp_pi = 0.14

phase_start= 0
phase_stop  = 1e-4
phase_points = 21
input_amp= 0.25
alfa = 1.05

LO_input_freq = 5e9

start_cooldown_time = 100 #in nanoseconds
qd_duration = 480
input_duration = 200 #in nanosecs
tilt_delay = 100
readout_duration = 2000
flip_delay = 40
reset_time = 1000 
sequence_duration = 2.25e4 - 19684
sequence_duration_2 = 2.25e4  - qd_duration - 16
sequence_duration_3  = 4.5e3  - qd_duration - 68
pi_amplitude = 0.185
amplitudes = [0,1]
#if_freq = 37.86e6 #37.85e6
zero_point = 0.0495
meas_point = 0.01
n_avg = 1000

input_phase = np.linspace(phase_start, phase_stop, phase_points)

delay_1 = 2.25e4 - 19684 - 40 +280
delay_2 = 2.25e4 - qd_duration - 16
delay_3 = 2.25e4 - qd_duration - 68

desired_delay = 2.25e4
def qua_prog(delay_1, delay_2, delay_3):
    with program() as hello_qua:
        # QUA variables declaration
        I = declare(fixed)
        Q = declare(fixed)
        I_st = declare_stream()
        Q_st = declare_stream()
        phase = declare(fixed)
        n = declare(int)
        flag = declare(bool)

        with for_(*from_array(phase, input_phase)):
            play("x180" * amp(0), "qubit")
            wait(delay_3 * u.ns)
            with for_(n, 0, n < n_avg, n+1):
                play("x180" * amp(0), "qubit")
                wait(delay_2 * u.ns)
                with for_each_(flag, [False, True]):
                    #### Qubit at |0>
                    reset_global_phase()
                    reset_frame("stimulus")
                    frame_rotation_2pi(phase, 'stimulus')
                    align("qubit","stimulus","fast_flux_line","dc_flux_line","dc_flux_line2","resonator")

                    #JDPD PREPARATION
                    play("const"*amp(vpp_0 * 4), "fast_flux_line")
                    play("const"*amp(zero_point* 4), "dc_flux_line")
                    play("const"*amp(zero_point * 4*alfa), "dc_flux_line2")

                    #Drive
                    wait(start_cooldown_time * u.ns, "qubit")
                    play("x180", "qubit", condition=flag)

                    #Stimulus
                    wait((start_cooldown_time + qd_duration)  * u.ns, "stimulus")
                    play("cw"*amp(input_amp*4), "stimulus")

                    #after play it means delay keeping the same value, than it plays for 16 ( the dur set in the config)
                    wait((start_cooldown_time + qd_duration + input_duration -16 - flip_delay) * u.ns, "fast_flux_line")
                    wait((start_cooldown_time + qd_duration +input_duration + tilt_delay -16) * u.ns, "dc_flux_line")
                    wait((start_cooldown_time + qd_duration + input_duration + tilt_delay - 16) * u.ns, "dc_flux_line2")

                    #play("cw"*amp(input_amp), "stimulus")
                    play("const"*amp((vpp_pi - vpp_0) * 4), "fast_flux_line")
                    play("const"*amp((meas_point - zero_point) * 4), "dc_flux_line")
                    play("const"*amp((meas_point - zero_point) * 4*alfa), "dc_flux_line2")

                    #wait((readout_duration + 50 - 16) * u.ns, "stimulus")
                    wait((readout_duration + flip_delay+ 50 - 16) * u.ns, "fast_flux_line")
                    wait((readout_duration -tilt_delay + 50 - 16) * u.ns, "dc_flux_line")
                    wait((readout_duration -tilt_delay+ 50 - 16) * u.ns, "dc_flux_line2")

                    # Measurement
                    wait((start_cooldown_time + input_duration + tilt_delay+16 + qd_duration) * u.ns, "resonator")  # Wait some time after setting the dc flux
                    # Measure using dual demodulation
                    measure(
                        "readout",
                        "resonator",
                        None,
                        dual_demod.full("cos", "out1", "sin", "out2", I),
                        dual_demod.full("minus_sin", "out1", "cos", "out2", Q),
                    )
                    align("fast_flux_line","dc_flux_line","dc_flux_line2","resonator")
                    # End of the sequence
                    ramp_to_zero("fast_flux_line")
                    ramp_to_zero("dc_flux_line")
                    ramp_to_zero("dc_flux_line2")
                    wait((thermalization_time - delay_1) * u.ns, "resonator")
                    # Save the data
                    save(I, I_st)
                    save(Q, Q_st)

        # Transfer the data from the FPGA to the CPU and perform some operations on the way (buffering, averaging, FFT...)
        with stream_processing():
            I_st.buffer(2).buffer(n_avg).buffer(len(input_phase)).save("I")
            Q_st.buffer(2).buffer(n_avg).buffer(len(input_phase)).save("Q")

    return hello_qua

simulation = False
if simulation:
    simulation_config = SimulationConfig(duration=600000//4)
    # Good values
    delay1 = desired_delay - 19684 -40
    delay2 = desired_delay - qd_duration - 16
    delay3 = desired_delay - qd_duration - 68
    # Initial guess
    # delay1 = desired_delay
    # delay2 = desired_delay - x180_len
    # delay3 = desired_delay - x180_len
    for i in range(2):
        job = qmm.simulate(config1, qua_prog(int(delay1), int(delay2), int(delay3)), simulation_config)
        samples = job.get_simulated_samples()# get the waveform report object
        waveform_report = job.get_simulated_waveform_report()

        slopes = np.where(np.abs(np.diff(samples.con1.analog["5"])) > 0)
        slopes_diff = np.diff(slopes[0][::3])
        delays = np.array([np.diff(slopes[0][::3])[i] for i in [0, 1, 3]]) - thermalization_time
        print(slopes_diff)
        stim_amp = samples.con1.analog["1"][slopes[0][::3]]
        print(stim_amp)

        delay1 += delays[0]
        delay2 -= np.cumsum(np.diff(delays))[0] - desired_delay
        delay3 -= np.cumsum(np.diff(delays))[1] - 2*desired_delay - (np.cumsum(np.diff(delays))[0] - desired_delay)
        print(f"delay_1 to add: {delays[0]} ns")
        print(f"delay_2 to add: {-(np.cumsum(np.diff(delays))[0] - desired_delay)} ns")
        print(f"delay_3: {-(np.cumsum(np.diff(delays))[1] - 2*desired_delay - (np.cumsum(np.diff(delays))[0] - desired_delay))} ns")

else:
    if_freqs = np.linspace(100e6,100e6,1)
    prob_if = []
    prob_if = []
    plot = False
    for if_freq in if_freqs:
    
        qm1 = qmm.open_qm(config1, close_other_machines= False) 
        qm1.set_intermediate_frequency(element ='stimulus', freq= if_freq)
        job = qm1.execute(qua_prog(int(delay_1), int(delay_2), int(delay_3)))
        results = fetching_tool(job, data_list=["I", "Q"])#, "I_e", "Q_e"])
        res = results.fetch_all()
        qm1.close()

        res2 = np.array(res)
        I_tot_all_states,Q_tot_all_states= np.array(res2)
        #I_tot = np.transpose(I_tot)
        #Q_tot = np.transpose(Q_tot)72
        max_separation_radians = []
        colors = plt.cm.RdBu(np.linspace(0,1,len(amplitudes)))
        
        for i in range(len(amplitudes)):
            I_tot = I_tot_all_states[:,:,i]
            Q_tot = Q_tot_all_states[:,:,i]
            thrs_I_tot= (np.max(I_tot) + np.min(I_tot))/2
            thrs_Q_tot = (np.max(Q_tot) + np.min(Q_tot))/2
            plt.figure(1)
            prob_I = []
            prob_Q = []
            for i,start_point in enumerate(input_phase):
                x = np.ones(n_avg) * start_point
            
                I = I_tot[i]
                Q = Q_tot[i]
                plt.figure(1)
                plt.title('Q')
                plt.plot(x,Q,'o',color = 'k')
                prob_Q.append(len(np.where(Q>thrs_Q_tot)[0])/len(Q))
                prob_I.append(len(np.where(I>thrs_I_tot)[0])/len(I))
            
                plt.figure(2)
                plt.title('I')
                plt.plot(x,I,'o',color = 'k')
                
                phase = signal.detrend(np.unwrap(np.angle(I + 1j * Q)))
                plt.figure(3)
                plt.plot(x,phase,'o',color = 'k')
                plt.legend()
            
            plt.figure(5)
            plt.title('I')
            plt.plot(input_phase, prob_I, 'o-')
        
            plt.figure(8)
            plt.title('Q')
            plt.plot(input_phase, prob_Q, 'o-')
        
        #index_min = np.argmin(prob_Q)
        #max_separation_radians.append(input_phase[index_min])
            #plt.axvline(x = 0.0213,color = 'r')
        prob_if.append((max(prob_I) + 1 - min(prob_I))/2)
            #prob_g_if.append((max(prob_g) + 1 - min(prob_g))/2)
            
        
        #plt.figure(5)
        #plt.plot(input_phase, prob_e, 'o-')
        #plt.plot(input_phase, prob_g, 'o-')
        #plt.pause(0.1)

        #prob_e_if.append((max(prob_e) + 1 - min(prob_e))/2)
        #prob_g_if.append((max(prob_g) + 1 - min(prob_g))/2)
        
        #plt.axvline(x = 0.0213,color = 'r')
        #for i,phase_point in enumerate(input_phase):